In [ ]:
import numpy as np

class LogisticRegression(object):
    """
    Logistic Regression Classifier training by Newton Method
    """

    def __init__(self, error: float = 0.7, max_epoch: int = 100):
        """
        :param error: float, if the distance between new weight and 
                      old weight is less than error, the process 
                      of traing will break.
        :param max_epoch: if training epoch >= max_epoch the process 
                          of traing will break.
        """
        self.error = error
        self.max_epoch = max_epoch
        self.weight = None
        self.sign = np.vectorize(lambda x: 1 if x >= 0.5 else 0)

    def p_func(self, X_): # 返回h(x)的计算结果
        """Get P(y=1 | x)
        :param X_: shape = (n_samples + 1, n_features)
        :return: shape = (n_samples)
        """
        tmp = np.exp(self.weight @ X_.T) # exp(wx+b)的结果
        return tmp / (1 + tmp) # 返回h(x)的计算结果

    def diff(self, X_, y, p): # 计算一阶导数
        """Get derivative
        :param X_: shape = (n_samples, n_features + 1) 
        :param y: shape = (n_samples)
        :param p: shape = (n_samples) P(y=1 | x)
        :return:  shape = (n_features + 1) first derivative
        """
        return -(y - p) @ X_ # (h(x) - y) * x为一阶导数

    def hess_mat(self, X_, p): # 返回海森矩阵
        """Get Hessian Matrix
        :param p: shape = (n_samples) P(y=1 | x)
        :return: shape = (n_features + 1, n_features + 1) second derivative
        """
        hess = np.zeros((X_.shape[1], X_.shape[1])) # 海森矩阵的维度为(n+1) * (n+1), n为样本特征维度
        for i in range(X_.shape[0]): # 对于每一个样本迭代一次
            hess += self.X_XT[i] * p[i] * (1 - p[i]) # self.X_XT[i]为样本i特征取值相乘生成的(n+1) * (n+1)矩阵 p[i]为标量
        return hess

    def newton_method(self, X_, y): # 牛顿法本体
        """Newton Method to calculate weight
        :param X_: shape = (n_samples + 1, n_features)
        :param y: shape = (n_samples)
        :return: None
        """
        self.weight = np.ones(X_.shape[1]) # 初始化权重
        self.X_XT = []
        for i in range(X_.shape[0]): # 对于每一行
            t = X_[i, :].reshape((-1, 1)) # 把每一行变成列向量
            self.X_XT.append(t @ t.T) # 相乘得到矩阵

        for _ in range(self.max_epoch): # 在最大迭代次数前
            p = self.p_func(X_) # 函数预测值
            diff = self.diff(X_, y, p) # 一阶导数
            hess = self.hess_mat(X_, p) # 海森矩阵
            new_weight = self.weight - (np.linalg.inv(hess) @ diff.reshape((-1, 1))).flatten() # 权重更新

            if np.linalg.norm(new_weight - self.weight) <= self.error: # 变化范围小于error
                break # 结束循环
            self.weight = new_weight

    def fit(self, X, y):
        """
        :param X_: shape = (n_samples, n_features)
        :param y: shape = (n_samples)
        :return: self
        """
        X_ = np.c_[np.ones(X.shape[0]), X] # 增广矩阵,相当于每一行多出一个1的列
        self.newton_method(X_, y)
        return self

    def predict(self, X) -> np.array:
        """
        :param X: shape = (n_samples, n_features] 
        :return: shape = (n_samples]
        """
        X_ = np.c_[np.ones(X.shape[0]), X]
        return self.sign(self.p_func(X_))